In [1]:
import numpy as np
import pandas as pd
import warnings
import calendar
warnings.filterwarnings('ignore')

In [2]:
# import libraties
import matplotlib.pyplot as plt
import seaborn as sns
#Suppressing scientific notation to float only 
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
from dateutil.parser import parse 
import matplotlib as mpl
plt.rcParams.update({'figure.figsize': (10, 7), 'figure.dpi': 120})

 ###  :::::::: Data Upload ::::::::

In [4]:
#Import the order data set
dfGam=pd.read_csv('GamingAccessory.csv')
dfGam.head()

,Unnamed: 0,order_date,Year,Month,order_id,order_item_id,gmv,units,deliverybdays,deliverycdays,...,inc_PO_MA3,LP_lag_1_per,LP_lag_2_per,LP_lag_3_per,PO_lag_1_per-1,PO_lag_1_per-2,PO_lag_1_per-3,GMV_lag_1_per,GMV_lag_2_per,GMV_lag_3_per
0,1555,2015-10-04,2015,10,"1,408,216,975,064,600.00","1,408,216,975,064,600.00","3,075.00",1,0,0,...,nan,0.12,0.06,0.18,-0.26,-0.16,-0.34,0.12,0.06,0.18
1,1556,2015-10-14,2015,10,"1,416,985,339,691,600.00","1,416,985,339,691,600.00","2,750.00",1,0,0,...,nan,-0.05,0.05,-0.01,0.14,-0.10,0.02,-0.05,0.05,-0.01
2,1557,2015-10-15,2015,10,"1,417,669,228,375,400.00","1,417,669,228,375,400.00","2,899.00",1,0,0,...,nan,0.11,0.05,0.05,-0.21,-0.10,-0.12,0.11,0.05,0.05
3,1558,2015-10-17,2015,10,"1,419,414,998,353,400.00","1,419,414,998,353,400.00","2,609.00",1,0,0,...,0.19,-0.06,-0.05,-0.13,0.13,0.11,0.39,-0.06,-0.05,-0.13
4,1559,2015-10-26,2015,10,"1,427,167,582,532,600.00","1,427,167,582,532,600.00","2,772.00",1,0,0,...,-0.01,0.01,-0.08,-0.10,-0.02,0.23,0.33,0.01,-0.08,-0.10


In [5]:
# Drop categorical variable columns
dfGam = dfGam.drop(['Unnamed: 0'], axis=1)
dfGam.head()

,order_date,Year,Month,order_id,order_item_id,gmv,units,deliverybdays,deliverycdays,s1_fact_order_payment_type,...,inc_PO_MA3,LP_lag_1_per,LP_lag_2_per,LP_lag_3_per,PO_lag_1_per-1,PO_lag_1_per-2,PO_lag_1_per-3,GMV_lag_1_per,GMV_lag_2_per,GMV_lag_3_per
0,2015-10-04,2015,10,"1,408,216,975,064,600.00","1,408,216,975,064,600.00","3,075.00",1,0,0,COD,...,nan,0.12,0.06,0.18,-0.26,-0.16,-0.34,0.12,0.06,0.18
1,2015-10-14,2015,10,"1,416,985,339,691,600.00","1,416,985,339,691,600.00","2,750.00",1,0,0,COD,...,nan,-0.05,0.05,-0.01,0.14,-0.10,0.02,-0.05,0.05,-0.01
2,2015-10-15,2015,10,"1,417,669,228,375,400.00","1,417,669,228,375,400.00","2,899.00",1,0,0,Prepaid,...,nan,0.11,0.05,0.05,-0.21,-0.10,-0.12,0.11,0.05,0.05
3,2015-10-17,2015,10,"1,419,414,998,353,400.00","1,419,414,998,353,400.00","2,609.00",1,0,0,Prepaid,...,0.19,-0.06,-0.05,-0.13,0.13,0.11,0.39,-0.06,-0.05,-0.13
4,2015-10-26,2015,10,"1,427,167,582,532,600.00","1,427,167,582,532,600.00","2,772.00",1,0,0,Prepaid,...,-0.01,0.01,-0.08,-0.10,-0.02,0.23,0.33,0.01,-0.08,-0.10


In [6]:
#dropping Unnamed: 0 column
dfGam.drop('week_y', axis=1, inplace=True)

In [7]:
dfGam.rename({' Affiliates':'Affiliates'},axis=1, inplace=True)

In [8]:
#check the dtypes
dfGam.dtypes

order_date                        object
Year                               int64
Month                              int64
order_id                         float64
order_item_id                    float64
gmv                              float64
units                              int64
deliverybdays                      int64
deliverycdays                      int64
s1_fact_order_payment_type        object
sla                                int64
product_analytic_category         object
product_analytic_sub_category     object
product_analytic_vertical         object
product_mrp                      float64
product_procurement_sla            int64
wday                               int64
week                               int64
Pay_Day                            int64
is_holiday                         int64
special_sale                      object
Discount_Percentage              float64
gmvperunit                       float64
delivery_on_time                 float64
delivery_status 

In [9]:
dfGam.shape

(170712, 65)

In [10]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression

In [11]:
# Summary of the dataset:
print(dfGam.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170712 entries, 0 to 170711
Data columns (total 65 columns):
order_date                       170712 non-null object
Year                             170712 non-null int64
Month                            170712 non-null int64
order_id                         170712 non-null float64
order_item_id                    170712 non-null float64
gmv                              170712 non-null float64
units                            170712 non-null int64
deliverybdays                    170712 non-null int64
deliverycdays                    170712 non-null int64
s1_fact_order_payment_type       170712 non-null object
sla                              170712 non-null int64
product_analytic_category        170712 non-null object
product_analytic_sub_category    170712 non-null object
product_analytic_vertical        170712 non-null object
product_mrp                      170712 non-null float64
product_procurement_sla          170712 non-null in

In [12]:
dfGam['delivery_on_time'].unique()

array([  4.,   2.,   0.,  -1.,   1.,   5.,   6.,   7.,   8.,   9.,  10.,
        13.,   3.,  11.,  16.,  14.,  -3.,  -2.,  17.,  15.,  12., -13.,
       -12.,  -9., -11.,  -8., -10.,  -7.,  -5.,  -4.,  -6.,  19.,  18.,
       -14.,  21., -15., -16., -61., -19.,  20.,  59., -20., -25., -22.,
       -24., -18., -17., -23., -21., -45., -28., -75., -64., -50., -33.,
       -36.,  24., -29., -27., -44., -39., -26., -34., -31., -30., -37.])

In [14]:
dfGam1=dfGam.loc[(dfGam['delivery_on_time']>0)]
dfGam1.head()

,order_date,Year,Month,order_id,order_item_id,gmv,units,deliverybdays,deliverycdays,s1_fact_order_payment_type,...,inc_PO_MA3,LP_lag_1_per,LP_lag_2_per,LP_lag_3_per,PO_lag_1_per-1,PO_lag_1_per-2,PO_lag_1_per-3,GMV_lag_1_per,GMV_lag_2_per,GMV_lag_3_per
0,2015-10-04,2015,10,"1,408,216,975,064,600.00","1,408,216,975,064,600.00","3,075.00",1,0,0,COD,...,nan,0.12,0.06,0.18,-0.26,-0.16,-0.34,0.12,0.06,0.18
1,2015-10-14,2015,10,"1,416,985,339,691,600.00","1,416,985,339,691,600.00","2,750.00",1,0,0,COD,...,nan,-0.05,0.05,-0.01,0.14,-0.10,0.02,-0.05,0.05,-0.01
5,2015-10-16,2015,10,"4,417,730,112,316,700.00","4,417,730,112,316,700.00","2,750.00",1,0,0,Prepaid,...,0.01,-0.08,-0.11,-0.05,0.25,0.36,0.14,-0.08,-0.11,-0.05
7,2015-10-24,2015,10,"1,424,897,937,886,500.00","1,424,897,937,886,500.00","3,079.00",1,0,0,Prepaid,...,-0.16,0.06,0.12,0.06,-0.16,-0.26,-0.16,0.06,0.12,0.06
8,2015-10-19,2015,10,"1,421,036,227,532,500.00","1,421,036,227,532,500.00","2,899.00",1,0,0,COD,...,0.03,0.05,0.00,-0.06,-0.12,0.00,0.20,0.05,0.00,-0.53


In [15]:
dfGam1.shape

(106810, 65)

In [16]:
log_cols = dfGam1.select_dtypes(['float64','int64']).columns

In [17]:
log_cols = list(set(log_cols)-
                set(['week','order_date','Year','Month', 'order_id', 'order_item_id', 'gmv',
                     'Share_of_Sales','Revenue','cust_id']))

In [18]:
# looking for any negative numbers as logs of negative numbers are not defined
dfGam1[log_cols].lt(0).sum()

inc_PO_MA1                   23822
GMV_lag_3_per                31689
Other                            0
Online marketing_adstock         0
PO_lag_1_per-1               24408
Sponsorship_adstock              0
PO_lag_1_per-2               28794
wday                             0
Radio                            0
inc_PO_MA3                   49045
Total Investment                 0
Affiliates_adstock               0
is_holiday                       0
sla                              0
NPS                              0
GMV_lag_1_per                25157
LP_lag_2_per                 28109
deliverycdays                    0
Other_adstock                    0
LP_lag_1_per                 23902
SEM_adstock                      0
deliverybdays                    0
TV                               0
Radio_adstock                    0
delivery_on_time                 0
Content Marketing                0
holiday_count                    0
product_mrp                      0
inc_LP_MA2          

In [19]:
# Drop categorical variable columns
dfGam1 = dfGam1.drop(['inc_LP_MA1','inc_LP_MA2','inc_LP_MA3','inc_PO_MA1','inc_PO_MA2','inc_PO_MA3','LP_lag_1_per',
           'LP_lag_2_per','LP_lag_3_per','PO_lag_1_per-1','PO_lag_1_per-2','PO_lag_1_per-1','GMV_lag_1_per','GMV_lag_2_per',
           'GMV_lag_3_per','PO_lag_1_per-3'], axis=1)
dfGam1.head()

,order_date,Year,Month,order_id,order_item_id,gmv,units,deliverybdays,deliverycdays,s1_fact_order_payment_type,...,month_date,TV_adstock,Digital_adstock,Sponsorship_adstock,Content Marketing_adstock,Online marketing_adstock,Affiliates_adstock,SEM_adstock,Radio_adstock,Other_adstock
0,2015-10-04,2015,10,"1,408,216,975,064,600.00","1,408,216,975,064,600.00","3,075.00",1,0,0,COD,...,2015-10-04-10,"13,875,154.09","28,502,373.67","191,196,038.93","7,776,943.00","55,033,046.50","15,747,089.60","72,093,251.50",0.00,0.00
1,2015-10-14,2015,10,"1,416,985,339,691,600.00","1,416,985,339,691,600.00","2,750.00",1,0,0,COD,...,2015-10-14-10,"20,812,731.13","42,753,560.50","286,794,058.40","11,665,414.50","82,549,569.75","23,620,634.40","108,139,877.25",0.00,0.00
5,2015-10-16,2015,10,"4,417,730,112,316,700.00","4,417,730,112,316,700.00","2,750.00",1,0,0,Prepaid,...,2015-10-16-10,"27,316,709.60","56,114,048.16","376,417,201.64","15,310,856.53","108,346,310.30","31,002,082.65","141,933,588.89",0.00,0.00
7,2015-10-24,2015,10,"1,424,897,937,886,500.00","1,424,897,937,886,500.00","3,079.00",1,0,0,Prepaid,...,2015-10-24-10,"27,641,908.53","56,782,072.54","380,898,358.81","15,493,128.63","109,636,147.33","31,371,155.06","143,623,274.48",0.00,0.00
8,2015-10-19,2015,10,"1,421,036,227,532,500.00","1,421,036,227,532,500.00","2,899.00",1,0,0,COD,...,2015-10-19-10,"27,696,108.35","56,893,409.94","381,645,218.33","15,523,507.31","109,851,120.17","31,432,667.13","143,904,888.74",0.00,0.00


In [20]:
log_cols = dfGam1.select_dtypes(['float64','int64']).columns

In [21]:
log_cols = list(set(log_cols)-
                set(['week','order_date','Year','Month', 'order_id','gmv',
                     'Share_of_Sales','Revenue']))

In [22]:
# looking for any negative numbers as logs of negative numbers are not defined
dfGam1[log_cols].lt(0).sum()

Other                        0
Online marketing_adstock     0
Sponsorship_adstock          0
wday                         0
Radio                        0
Total Investment             0
Affiliates_adstock           0
order_item_id                0
is_holiday                   0
sla                          0
NPS                          0
deliverycdays                0
Other_adstock                0
SEM_adstock                  0
deliverybdays                0
TV                           0
Radio_adstock                0
delivery_on_time             0
Content Marketing            0
holiday_count                0
product_mrp                  0
gmvperunit                   0
SEM                          0
Digital                      0
Affiliates                   0
NPSScore                     0
product_procurement_sla      0
Online marketing             0
Content Marketing_adstock    0
TV_adstock                   0
units                        0
Stock Index                  0
Digital_

In [23]:
dfGam1[log_cols]+=1

In [24]:
# Transforming relevant columns to logs
dfGam1[log_cols] =np.log(dfGam1[log_cols])

In [25]:
dfGam1.head()

,order_date,Year,Month,order_id,order_item_id,gmv,units,deliverybdays,deliverycdays,s1_fact_order_payment_type,...,month_date,TV_adstock,Digital_adstock,Sponsorship_adstock,Content Marketing_adstock,Online marketing_adstock,Affiliates_adstock,SEM_adstock,Radio_adstock,Other_adstock
0,2015-10-04,2015,10,"1,408,216,975,064,600.00",34.88,"3,075.00",0.69,0.00,0.00,COD,...,2015-10-04-10,16.45,17.17,19.07,15.87,17.82,16.57,18.09,0.00,0.00
1,2015-10-14,2015,10,"1,416,985,339,691,600.00",34.89,"2,750.00",0.69,0.00,0.00,COD,...,2015-10-14-10,16.85,17.57,19.47,16.27,18.23,16.98,18.50,0.00,0.00
5,2015-10-16,2015,10,"4,417,730,112,316,700.00",36.02,"2,750.00",0.69,0.00,0.00,Prepaid,...,2015-10-16-10,17.12,17.84,19.75,16.54,18.50,17.25,18.77,0.00,0.00
7,2015-10-24,2015,10,"1,424,897,937,886,500.00",34.89,"3,079.00",0.69,0.00,0.00,Prepaid,...,2015-10-24-10,17.13,17.85,19.76,16.56,18.51,17.26,18.78,0.00,0.00
8,2015-10-19,2015,10,"1,421,036,227,532,500.00",34.89,"2,899.00",0.69,0.00,0.00,COD,...,2015-10-19-10,17.14,17.86,19.76,16.56,18.51,17.26,18.78,0.00,0.00


In [26]:
# Subset all categorical variables
ecart_GA_numerical = dfGam1.select_dtypes(include=['int64','float64'])
ecart_GA_numerical.head()

,Year,Month,order_id,order_item_id,gmv,units,deliverybdays,deliverycdays,sla,product_mrp,...,holiday_count,TV_adstock,Digital_adstock,Sponsorship_adstock,Content Marketing_adstock,Online marketing_adstock,Affiliates_adstock,SEM_adstock,Radio_adstock,Other_adstock
0,2015,10,"1,408,216,975,064,600.00",34.88,"3,075.00",0.69,0.00,0.00,2.30,8.29,...,0.00,16.45,17.17,19.07,15.87,17.82,16.57,18.09,0.00,0.00
1,2015,10,"1,416,985,339,691,600.00",34.89,"2,750.00",0.69,0.00,0.00,2.08,8.29,...,1.61,16.85,17.57,19.47,16.27,18.23,16.98,18.50,0.00,0.00
5,2015,10,"4,417,730,112,316,700.00",36.02,"2,750.00",0.69,0.00,0.00,2.08,8.29,...,1.61,17.12,17.84,19.75,16.54,18.50,17.25,18.77,0.00,0.00
7,2015,10,"1,424,897,937,886,500.00",34.89,"3,079.00",0.69,0.00,0.00,2.08,8.29,...,0.00,17.13,17.85,19.76,16.56,18.51,17.26,18.78,0.00,0.00
8,2015,10,"1,421,036,227,532,500.00",34.89,"2,899.00",0.69,0.00,0.00,1.95,8.29,...,0.00,17.14,17.86,19.76,16.56,18.51,17.26,18.78,0.00,0.00


In [27]:
ecart_GA_numerical.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106810 entries, 0 to 170711
Data columns (total 41 columns):
Year                         106810 non-null int64
Month                        106810 non-null int64
order_id                     106810 non-null float64
order_item_id                106810 non-null float64
gmv                          106810 non-null float64
units                        106810 non-null float64
deliverybdays                106810 non-null float64
deliverycdays                106810 non-null float64
sla                          106810 non-null float64
product_mrp                  106810 non-null float64
product_procurement_sla      106810 non-null float64
wday                         106810 non-null float64
week                         106810 non-null int64
Pay_Day                      106810 non-null float64
is_holiday                   106810 non-null float64
Discount_Percentage          106810 non-null float64
gmvperunit                   106810 non-null fl

In [28]:
# Drop categorical variable columns
dfGam1 = dfGam1.drop(['Year','Month','order_id','order_item_id','order_date'], axis=1)
dfGam1.head()

,gmv,units,deliverybdays,deliverycdays,s1_fact_order_payment_type,sla,product_analytic_category,product_analytic_sub_category,product_analytic_vertical,product_mrp,...,month_date,TV_adstock,Digital_adstock,Sponsorship_adstock,Content Marketing_adstock,Online marketing_adstock,Affiliates_adstock,SEM_adstock,Radio_adstock,Other_adstock
0,"3,075.00",0.69,0.00,0.00,COD,2.30,GamingHardware,GamingAccessory,GamePad,8.29,...,2015-10-04-10,16.45,17.17,19.07,15.87,17.82,16.57,18.09,0.00,0.00
1,"2,750.00",0.69,0.00,0.00,COD,2.08,GamingHardware,GamingAccessory,GamePad,8.29,...,2015-10-14-10,16.85,17.57,19.47,16.27,18.23,16.98,18.50,0.00,0.00
5,"2,750.00",0.69,0.00,0.00,Prepaid,2.08,GamingHardware,GamingAccessory,GamePad,8.29,...,2015-10-16-10,17.12,17.84,19.75,16.54,18.50,17.25,18.77,0.00,0.00
7,"3,079.00",0.69,0.00,0.00,Prepaid,2.08,GamingHardware,GamingAccessory,GamePad,8.29,...,2015-10-24-10,17.13,17.85,19.76,16.56,18.51,17.26,18.78,0.00,0.00
8,"2,899.00",0.69,0.00,0.00,COD,1.95,GamingHardware,GamingAccessory,GamePad,8.29,...,2015-10-19-10,17.14,17.86,19.76,16.56,18.51,17.26,18.78,0.00,0.00


In [29]:
# Subset all categorical variables
ecart_GA_categorical = dfGam1.select_dtypes(include=['object'])
ecart_GA_categorical.head()

,s1_fact_order_payment_type,product_analytic_category,product_analytic_sub_category,product_analytic_vertical,special_sale,delivery_status,ItemType,month_date
0,COD,GamingHardware,GamingAccessory,GamePad,Regular Day,Early,Luxury,2015-10-04-10
1,COD,GamingHardware,GamingAccessory,GamePad,Regular Day,Early,Luxury,2015-10-14-10
5,Prepaid,GamingHardware,GamingAccessory,GamePad,Daussera,Early,Luxury,2015-10-16-10
7,Prepaid,GamingHardware,GamingAccessory,GamePad,Regular Day,Early,Luxury,2015-10-24-10
8,COD,GamingHardware,GamingAccessory,GamePad,Regular Day,Early,Luxury,2015-10-19-10


In [30]:
# Drop categorical variable columns
dfGam1 = dfGam1.drop(['month_date'], axis=1)
dfGam1.head()

,gmv,units,deliverybdays,deliverycdays,s1_fact_order_payment_type,sla,product_analytic_category,product_analytic_sub_category,product_analytic_vertical,product_mrp,...,ItemType,TV_adstock,Digital_adstock,Sponsorship_adstock,Content Marketing_adstock,Online marketing_adstock,Affiliates_adstock,SEM_adstock,Radio_adstock,Other_adstock
0,"3,075.00",0.69,0.00,0.00,COD,2.30,GamingHardware,GamingAccessory,GamePad,8.29,...,Luxury,16.45,17.17,19.07,15.87,17.82,16.57,18.09,0.00,0.00
1,"2,750.00",0.69,0.00,0.00,COD,2.08,GamingHardware,GamingAccessory,GamePad,8.29,...,Luxury,16.85,17.57,19.47,16.27,18.23,16.98,18.50,0.00,0.00
5,"2,750.00",0.69,0.00,0.00,Prepaid,2.08,GamingHardware,GamingAccessory,GamePad,8.29,...,Luxury,17.12,17.84,19.75,16.54,18.50,17.25,18.77,0.00,0.00
7,"3,079.00",0.69,0.00,0.00,Prepaid,2.08,GamingHardware,GamingAccessory,GamePad,8.29,...,Luxury,17.13,17.85,19.76,16.56,18.51,17.26,18.78,0.00,0.00
8,"2,899.00",0.69,0.00,0.00,COD,1.95,GamingHardware,GamingAccessory,GamePad,8.29,...,Luxury,17.14,17.86,19.76,16.56,18.51,17.26,18.78,0.00,0.00


In [31]:
# Subset all categorical variables
ecart_GA_categorical = dfGam1.select_dtypes(include=['object'])
ecart_GA_categorical.head()

,s1_fact_order_payment_type,product_analytic_category,product_analytic_sub_category,product_analytic_vertical,special_sale,delivery_status,ItemType
0,COD,GamingHardware,GamingAccessory,GamePad,Regular Day,Early,Luxury
1,COD,GamingHardware,GamingAccessory,GamePad,Regular Day,Early,Luxury
5,Prepaid,GamingHardware,GamingAccessory,GamePad,Daussera,Early,Luxury
7,Prepaid,GamingHardware,GamingAccessory,GamePad,Regular Day,Early,Luxury
8,COD,GamingHardware,GamingAccessory,GamePad,Regular Day,Early,Luxury


In [32]:
# Subset all categorical variables
ecart_GA_numerical = dfGam1.select_dtypes(include=['int64','float64'])
ecart_GA_numerical.head()

,gmv,units,deliverybdays,deliverycdays,sla,product_mrp,product_procurement_sla,wday,week,Pay_Day,...,holiday_count,TV_adstock,Digital_adstock,Sponsorship_adstock,Content Marketing_adstock,Online marketing_adstock,Affiliates_adstock,SEM_adstock,Radio_adstock,Other_adstock
0,"3,075.00",0.69,0.00,0.00,2.30,8.29,1.79,1.95,40,0.00,...,0.00,16.45,17.17,19.07,15.87,17.82,16.57,18.09,0.00,0.00
1,"2,750.00",0.69,0.00,0.00,2.08,8.29,1.79,1.10,42,0.00,...,1.61,16.85,17.57,19.47,16.27,18.23,16.98,18.50,0.00,0.00
5,"2,750.00",0.69,0.00,0.00,2.08,8.29,1.79,1.61,42,0.00,...,1.61,17.12,17.84,19.75,16.54,18.50,17.25,18.77,0.00,0.00
7,"3,079.00",0.69,0.00,0.00,2.08,8.29,1.79,1.79,43,0.00,...,0.00,17.13,17.85,19.76,16.56,18.51,17.26,18.78,0.00,0.00
8,"2,899.00",0.69,0.00,0.00,1.95,8.29,1.79,0.00,43,0.00,...,0.00,17.14,17.86,19.76,16.56,18.51,17.26,18.78,0.00,0.00


In [33]:
ecart_GA_numerical.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106810 entries, 0 to 170711
Data columns (total 37 columns):
gmv                          106810 non-null float64
units                        106810 non-null float64
deliverybdays                106810 non-null float64
deliverycdays                106810 non-null float64
sla                          106810 non-null float64
product_mrp                  106810 non-null float64
product_procurement_sla      106810 non-null float64
wday                         106810 non-null float64
week                         106810 non-null int64
Pay_Day                      106810 non-null float64
is_holiday                   106810 non-null float64
Discount_Percentage          106810 non-null float64
gmvperunit                   106810 non-null float64
delivery_on_time             106810 non-null float64
Total Investment             106810 non-null float64
TV                           106810 non-null float64
Digital                      106810 non-nul

In [34]:
# Convert into dummies
ecart_GA_dummies = pd.get_dummies(ecart_GA_categorical, drop_first=True)
ecart_GA_dummies.head()

,s1_fact_order_payment_type_Prepaid,product_analytic_vertical_GameControlMount,product_analytic_vertical_GamePad,product_analytic_vertical_GamingAccessoryKit,product_analytic_vertical_GamingAdapter,product_analytic_vertical_GamingChargingStation,product_analytic_vertical_GamingHeadset,product_analytic_vertical_GamingKeyboard,product_analytic_vertical_GamingMemoryCard,product_analytic_vertical_GamingMouse,...,special_sale_Christmas & New Year,special_sale_Daussera,special_sale_Diwali,special_sale_Eid & Rathayatra,special_sale_FHSD,special_sale_Pacman,special_sale_Regular Day,special_sale_Republic Day,special_sale_Valentine Day,ItemType_MassMarket
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
5,1,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
7,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
8,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [35]:
#dfcam1[ecart_CA_dummies].lt(0).sum()
#missing information
ecart_GA_dummies.isnull().any()

s1_fact_order_payment_type_Prepaid                 False
product_analytic_vertical_GameControlMount         False
product_analytic_vertical_GamePad                  False
product_analytic_vertical_GamingAccessoryKit       False
product_analytic_vertical_GamingAdapter            False
product_analytic_vertical_GamingChargingStation    False
product_analytic_vertical_GamingHeadset            False
product_analytic_vertical_GamingKeyboard           False
product_analytic_vertical_GamingMemoryCard         False
product_analytic_vertical_GamingMouse              False
product_analytic_vertical_GamingMousePad           False
product_analytic_vertical_GamingSpeaker            False
product_analytic_vertical_JoystickGamingWheel      False
product_analytic_vertical_MotionController         False
product_analytic_vertical_TVOutCableAccessory      False
special_sale_BSD                                   False
special_sale_Christmas & New Year                  False
special_sale_Daussera          

In [36]:
ecart_GA_dummies['s1_fact_order_payment_type_Prepaid'] = ecart_GA_dummies['s1_fact_order_payment_type_Prepaid'].replace(['0'], '0.0001')
ecart_GA_dummies['product_analytic_vertical_GameControlMount'] = ecart_GA_dummies['product_analytic_vertical_GameControlMount'].replace(['0'], '0.0001')
ecart_GA_dummies['product_analytic_vertical_GamePad'] = ecart_GA_dummies['product_analytic_vertical_GamePad'].replace(['0'], '0.0001')
ecart_GA_dummies['product_analytic_vertical_GamingAccessoryKit'] = ecart_GA_dummies['product_analytic_vertical_GamingAccessoryKit'].replace(['0'], '0.0001')
ecart_GA_dummies['product_analytic_vertical_GamingAdapter'] = ecart_GA_dummies['product_analytic_vertical_GamingAdapter'].replace(['0'], '0.0001')
ecart_GA_dummies['product_analytic_vertical_GamingChargingStation'] = ecart_GA_dummies['product_analytic_vertical_GamingChargingStation'].replace(['0'], '0.0001')
ecart_GA_dummies['product_analytic_vertical_GamingHeadset'] = ecart_GA_dummies['product_analytic_vertical_GamingHeadset'].replace(['0'], '0.0001')
ecart_GA_dummies['product_analytic_vertical_GamingKeyboard'] = ecart_GA_dummies['product_analytic_vertical_GamingKeyboard'].replace(['0'], '0.0001')
ecart_GA_dummies['product_analytic_vertical_GamingMemoryCard'] = ecart_GA_dummies['product_analytic_vertical_GamingMemoryCard'].replace(['0'], '0.0001')
ecart_GA_dummies['product_analytic_vertical_GamingMouse'] = ecart_GA_dummies['product_analytic_vertical_GamingMouse'].replace(['0'], '0.0001')

In [37]:
ecart_GA_dummies['product_analytic_vertical_GamingMousePad'] = ecart_GA_dummies['product_analytic_vertical_GamingMousePad'].replace(['0'], '0.0001')
ecart_GA_dummies['product_analytic_vertical_GamingSpeaker'] = ecart_GA_dummies['product_analytic_vertical_GamingSpeaker'].replace(['0'], '0.0001')
ecart_GA_dummies['product_analytic_vertical_JoystickGamingWheel'] = ecart_GA_dummies['product_analytic_vertical_JoystickGamingWheel'].replace(['0'], '0.0001')
ecart_GA_dummies['product_analytic_vertical_TVOutCableAccessory'] = ecart_GA_dummies['product_analytic_vertical_TVOutCableAccessory'].replace(['0'], '0.0001')
ecart_GA_dummies['special_sale_BSD'] = ecart_GA_dummies['special_sale_BSD'].replace(['0'], '0.0001')
ecart_GA_dummies['special_sale_Christmas & New Year'] = ecart_GA_dummies['special_sale_Christmas & New Year'].replace(['0'], '0.0001')
ecart_GA_dummies['special_sale_Daussera'] = ecart_GA_dummies['special_sale_Daussera'].replace(['0'], '0.0001')
ecart_GA_dummies['special_sale_Diwali'] = ecart_GA_dummies['special_sale_Diwali'].replace(['0'], '0.0001')
ecart_GA_dummies['special_sale_Eid & Rathayatra'] = ecart_GA_dummies['special_sale_Eid & Rathayatra'].replace(['0'], '0.0001')
ecart_GA_dummies['special_sale_FHSD'] = ecart_GA_dummies['special_sale_FHSD'].replace(['0'], '0.0001')

In [38]:
ecart_GA_dummies['special_sale_Pacman'] = ecart_GA_dummies['special_sale_Pacman'].replace(['0'], '0.0001')
ecart_GA_dummies['special_sale_Regular Day'] = ecart_GA_dummies['special_sale_Regular Day'].replace(['0'], '0.0001')
ecart_GA_dummies['special_sale_Republic Day'] = ecart_GA_dummies['special_sale_Republic Day'].replace(['0'], '0.0001')
ecart_GA_dummies['special_sale_Valentine Day'] = ecart_GA_dummies['special_sale_Valentine Day'].replace(['0'], '0.0001')
ecart_GA_dummies['ItemType_MassMarket'] = ecart_GA_dummies['ItemType_MassMarket'].replace(['0'], '0.0001')

In [39]:
# Drop categorical variable columns
dfGam1 = dfGam1.drop(list(ecart_GA_categorical.columns), axis=1)

In [40]:
dfGam1.head()

,gmv,units,deliverybdays,deliverycdays,sla,product_mrp,product_procurement_sla,wday,week,Pay_Day,...,holiday_count,TV_adstock,Digital_adstock,Sponsorship_adstock,Content Marketing_adstock,Online marketing_adstock,Affiliates_adstock,SEM_adstock,Radio_adstock,Other_adstock
0,"3,075.00",0.69,0.00,0.00,2.30,8.29,1.79,1.95,40,0.00,...,0.00,16.45,17.17,19.07,15.87,17.82,16.57,18.09,0.00,0.00
1,"2,750.00",0.69,0.00,0.00,2.08,8.29,1.79,1.10,42,0.00,...,1.61,16.85,17.57,19.47,16.27,18.23,16.98,18.50,0.00,0.00
5,"2,750.00",0.69,0.00,0.00,2.08,8.29,1.79,1.61,42,0.00,...,1.61,17.12,17.84,19.75,16.54,18.50,17.25,18.77,0.00,0.00
7,"3,079.00",0.69,0.00,0.00,2.08,8.29,1.79,1.79,43,0.00,...,0.00,17.13,17.85,19.76,16.56,18.51,17.26,18.78,0.00,0.00
8,"2,899.00",0.69,0.00,0.00,1.95,8.29,1.79,0.00,43,0.00,...,0.00,17.14,17.86,19.76,16.56,18.51,17.26,18.78,0.00,0.00


In [41]:
# Concatenate dummy variables with X
ecart_GA = pd.concat([dfGam1, ecart_GA_dummies], axis=1)
ecart_GA.head()

,gmv,units,deliverybdays,deliverycdays,sla,product_mrp,product_procurement_sla,wday,week,Pay_Day,...,special_sale_Christmas & New Year,special_sale_Daussera,special_sale_Diwali,special_sale_Eid & Rathayatra,special_sale_FHSD,special_sale_Pacman,special_sale_Regular Day,special_sale_Republic Day,special_sale_Valentine Day,ItemType_MassMarket
0,"3,075.00",0.69,0.00,0.00,2.30,8.29,1.79,1.95,40,0.00,...,0,0,0,0,0,0,1,0,0,0
1,"2,750.00",0.69,0.00,0.00,2.08,8.29,1.79,1.10,42,0.00,...,0,0,0,0,0,0,1,0,0,0
5,"2,750.00",0.69,0.00,0.00,2.08,8.29,1.79,1.61,42,0.00,...,0,1,0,0,0,0,0,0,0,0
7,"3,079.00",0.69,0.00,0.00,2.08,8.29,1.79,1.79,43,0.00,...,0,0,0,0,0,0,1,0,0,0
8,"2,899.00",0.69,0.00,0.00,1.95,8.29,1.79,0.00,43,0.00,...,0,0,0,0,0,0,1,0,0,0


In [42]:
ecart_GA.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106810 entries, 0 to 170711
Data columns (total 63 columns):
gmv                                                106810 non-null float64
units                                              106810 non-null float64
deliverybdays                                      106810 non-null float64
deliverycdays                                      106810 non-null float64
sla                                                106810 non-null float64
product_mrp                                        106810 non-null float64
product_procurement_sla                            106810 non-null float64
wday                                               106810 non-null float64
week                                               106810 non-null int64
Pay_Day                                            106810 non-null float64
is_holiday                                         106810 non-null float64
Discount_Percentage                                106810 non-null

In [43]:
ecart_GA.shape

(106810, 63)

In [44]:
# Import the required library

from sklearn.model_selection import train_test_split

In [45]:
df_train, df_test = train_test_split(ecart_GA, train_size=0.7, test_size=0.3, random_state=100)

In [46]:
# Import MinMax scaler

from sklearn.preprocessing import MinMaxScaler

In [47]:
scaler = MinMaxScaler()


In [48]:
#ecart_CA_numerical.info()

In [49]:
# Create a list of the variables that you need to scale
varlist1 = ['gmv','product_mrp']

In [50]:

# Scale these variables using 'fit_transform'
df_train[varlist1] = scaler.fit_transform(df_train[varlist1])

In [51]:
# Let's take a look at the train dataframe now
df_train.head()

,gmv,units,deliverybdays,deliverycdays,sla,product_mrp,product_procurement_sla,wday,week,Pay_Day,...,special_sale_Christmas & New Year,special_sale_Daussera,special_sale_Diwali,special_sale_Eid & Rathayatra,special_sale_FHSD,special_sale_Pacman,special_sale_Regular Day,special_sale_Republic Day,special_sale_Valentine Day,ItemType_MassMarket
36102,0.01,0.69,0.00,0.00,2.08,0.43,1.79,0.00,51,0.00,...,0,0,0,0,0,0,1,0,0,1
98899,0.02,0.69,0.00,0.00,1.61,0.43,0.00,0.69,10,0.00,...,0,0,0,0,0,0,0,0,0,1
93860,0.01,1.10,0.00,0.00,1.95,0.19,0.69,0.69,12,0.00,...,0,0,0,0,0,0,1,0,0,1
637,0.01,0.69,0.00,0.00,2.20,0.43,1.79,1.61,41,0.00,...,0,0,0,0,0,0,1,0,0,1
41231,0.01,0.69,0.00,0.00,1.95,0.40,0.69,1.61,52,0.00,...,1,0,0,0,0,0,0,0,0,1


In [52]:
# Split the train dataset into X and y

y_train = df_train.pop('gmv')
X_train = df_train

In [53]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression

In [54]:
# Instantiate
lm = LinearRegression()

# Fit a line
lm.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [55]:
# Print the coefficients and intercept
print(lm.coef_)
print(lm.intercept_)

[ 3.54937306e-02  1.88301394e-04 -7.60813996e-04 -9.18514613e-04
  4.32909912e-02  9.82387950e-04 -9.61644722e-05  1.31091220e-05
  4.32027933e-04 -1.08390878e-03 -2.45211879e-03  9.14656174e-03
  8.07347522e-04  1.23381392e-02 -4.00336841e-03  4.30008884e-03
 -3.87200881e-04 -1.20123044e-03  1.47258253e-03 -1.13082224e-02
  4.25377096e-03 -3.76707150e-03  4.41787332e-03 -5.40520328e-03
  3.97710728e-03 -7.08645122e-03  2.42172174e-04  9.17418831e-04
 -1.44236618e-04 -8.53460193e-04  4.26066172e-05 -1.50533944e-02
  1.16809038e-02  2.62154974e-03 -1.29903232e-05  2.42927338e-05
 -4.48456137e-06  9.61767000e-03  2.94684464e-03  4.74043165e-03
  3.25972301e-03  1.45888893e-03  1.67427781e-03  2.23482750e-03
  3.33147325e-04  2.60151418e-03  1.37683514e-04 -2.85710294e-03
  2.28272072e-02  1.89185005e-02  5.10632777e-03 -4.35594196e-04
 -1.19401738e-04  1.01638550e-03  9.30015463e-04  1.19855488e-03
  9.63891720e-04 -1.19656745e-03  2.50799159e-05  2.49153185e-04
  7.58012522e-04 -3.55823

In [56]:
# Import RFE
from sklearn.feature_selection import RFE

# RFE with 15 features
lm = LinearRegression()
rfe1 = RFE(lm, 15)

# Fit with 15 features
rfe1.fit(X_train, y_train)

# Print the boolean results
print(rfe1.support_)           
print(rfe1.ranking_)  

[ True False False False  True False False False False False  True  True
 False  True False False False False False False False False False  True
  True  True False False False False False False False False False False
 False  True False  True False False False False False False False  True
  True  True  True False False False False False False False False False
 False  True]
[ 1 35 32 18  1 17 42 46 34 31  1  1 19  1  7 23 28 22  6  3  5  4 21  1
  1  1 36 20 45 24 39  2  9 15 47 44 48  1 11  1 10 16 14 13 37 12 38  1
  1  1  1 33 41 25 27 26 29  8 43 40 30  1]


In [57]:
# Import statsmodels
import statsmodels.api as sm  

# Subset the features selected by rfe1
col1 = X_train.columns[rfe1.support_]

# Subsetting training data for 15 selected columns
X_train_rfe1 = X_train[col1]

# Add a constant to the model
X_train_rfe1 = sm.add_constant(X_train_rfe1)
X_train_rfe1.head()

,const,units,product_mrp,Discount_Percentage,gmvperunit,Total Investment,NPS,Stock Index,NPSScore,product_analytic_vertical_GameControlMount,product_analytic_vertical_GamingAccessoryKit,product_analytic_vertical_GamingSpeaker,product_analytic_vertical_JoystickGamingWheel,product_analytic_vertical_MotionController,product_analytic_vertical_TVOutCableAccessory,ItemType_MassMarket
36102,1.00,0.69,0.43,4.24,6.31,19.30,3.85,6.95,2.43,0,0,0,0,0,0,1
98899,1.00,0.69,0.43,3.82,6.91,19.24,3.91,6.92,2.49,0,0,0,0,0,0,1
93860,1.00,1.10,0.19,3.50,5.63,19.24,3.91,6.92,2.49,0,0,0,0,0,0,1
637,1.00,0.69,0.43,4.26,6.27,19.77,3.82,7.10,2.40,0,0,0,0,0,0,1
41231,1.00,0.69,0.40,4.10,6.42,19.30,3.85,6.95,2.43,0,0,0,1,0,0,1


In [58]:
# Fitting the model with 15 variables
lm1 = sm.OLS(y_train, X_train_rfe1).fit()   
print(lm1.summary())

                            OLS Regression Results                            
Dep. Variable:                    gmv   R-squared:                       0.687
Model:                            OLS   Adj. R-squared:                  0.687
Method:                 Least Squares   F-statistic:                 1.095e+04
Date:                Sun, 15 Sep 2019   Prob (F-statistic):               0.00
Time:                        22:20:28   Log-Likelihood:             2.1789e+05
No. Observations:               74767   AIC:                        -4.358e+05
Df Residuals:                   74751   BIC:                        -4.356e+05
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                                                    coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------

In [59]:
# Check for the VIF values of the feature variables. 
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [60]:
# Creating a function for calculating VIF
def vif_calc(input_data):
    vif = pd.DataFrame()
    X = input_data
    vif['Features'] = X.columns
    vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by = "VIF", ascending = False)
    return vif

In [61]:
vif_calc(input_data=X_train_rfe1)

,Features,VIF
0,const,"94,782.39"
6,NPS,16.90
8,NPSScore,14.65
4,gmvperunit,6.03
2,product_mrp,5.76
5,Total Investment,4.12
3,Discount_Percentage,2.33
7,Stock Index,1.80
15,ItemType_MassMarket,1.59
13,product_analytic_vertical_MotionController,1.04


In [62]:
X_train_rfe1.drop('gmvperunit', axis = 1, inplace = True)

In [63]:
lm_2 = sm.OLS(y_train,X_train_rfe1).fit() 
lm_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    gmv   R-squared:                       0.667
Model:                            OLS   Adj. R-squared:                  0.667
Method:                 Least Squares   F-statistic:                 1.071e+04
Date:                Sun, 15 Sep 2019   Prob (F-statistic):               0.00
Time:                        22:20:30   Log-Likelihood:             2.1559e+05
No. Observations:               74767   AIC:                        -4.311e+05
Df Residuals:                   74752   BIC:                        -4.310e+05
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
const                                            -0.0976      0.015     -6.409      0.000      -0.127      -0.068
units                                             0.0353      0.001     58.145      0.000       0.034       0.037
product_mrp                                       0.0964      0.001    186.020      0.000       0.095       0.097
Discount_Percentage                              -0.0075   7.52e-05    -99.439      0.000      -0.008      -0.007
Total Investment                                  0.0004      0.000      2.152      0.031    3.45e-05       0.001
NPS                                               0.0232      0.004      6.147      0.000       0.016       0.031
Stock Index                                       0.0096      0.001     11.675      0.000       0.008       0.011
NPSScore                                         -0.0189      0.003     -6.290      0.000      -0.025      -0.013
product_analytic_vertical_GameControlMount        0.0115      0.014      0.847      0.397      -0.015       0.038
product_analytic_vertical_GamingAccessoryKit      0.0037      0.000     16.566      0.000       0.003       0.004
product_analytic_vertical_GamingSpeaker          -0.0050      0.001     -4.848      0.000      -0.007      -0.003
product_analytic_vertical_JoystickGamingWheel     0.0217      0.001     41.432      0.000       0.021       0.023
product_analytic_vertical_MotionController        0.0169      0.001     22.808      0.000       0.015       0.018
product_analytic_vertical_TVOutCableAccessory     0.0038      0.000     14.550      0.000       0.003       0.004
ItemType_MassMarket                              -0.0384      0.000   -142.892      0.000      -0.039      -0.038
==============================================================================
Omnibus:                   192290.141   Durbin-Watson:                   2.004
Prob(Omnibus):                  0.000   Jarque-Bera (JB):       7119751527.068
Skew:                          29.180   Prob(JB):                         0.00
Kurtosis:                    1513.634   Cond. No.                     6.70e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.7e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [64]:
vif_calc(input_data=X_train_rfe1)

,Features,VIF
0,const,"94,647.04"
5,NPS,16.89
7,NPSScore,14.63
4,Total Investment,4.12
6,Stock Index,1.79
2,product_mrp,1.60
14,ItemType_MassMarket,1.57
3,Discount_Percentage,1.16
12,product_analytic_vertical_MotionController,1.04
13,product_analytic_vertical_TVOutCableAccessory,1.03


In [65]:

X_train_rfe1 = X_train_rfe1.drop('NPS', axis=1)

lm_3 = sm.OLS(y_train,X_train_rfe1).fit() 
lm_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    gmv   R-squared:                       0.667
Model:                            OLS   Adj. R-squared:                  0.667
Method:                 Least Squares   F-statistic:                 1.153e+04
Date:                Sun, 15 Sep 2019   Prob (F-statistic):               0.00
Time:                        22:20:31   Log-Likelihood:             2.1557e+05
No. Observations:               74767   AIC:                        -4.311e+05
Df Residuals:                   74753   BIC:                        -4.310e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
const                                            -0.0163      0.008     -2.157      0.031      -0.031      -0.001
units                                             0.0354      0.001     58.226      0.000       0.034       0.037
product_mrp                                       0.0965      0.001    186.144      0.000       0.095       0.098
Discount_Percentage                              -0.0075   7.51e-05    -99.898      0.000      -0.008      -0.007
Total Investment                                 -0.0001      0.000     -0.778      0.437      -0.000       0.000
Stock Index                                       0.0065      0.001      9.994      0.000       0.005       0.008
NPSScore                                         -0.0026      0.001     -1.854      0.064      -0.005       0.000
product_analytic_vertical_GameControlMount        0.0112      0.014      0.831      0.406      -0.015       0.038
product_analytic_vertical_GamingAccessoryKit      0.0037      0.000     16.708      0.000       0.003       0.004
product_analytic_vertical_GamingSpeaker          -0.0050      0.001     -4.844      0.000      -0.007      -0.003
product_analytic_vertical_JoystickGamingWheel     0.0219      0.001     41.799      0.000       0.021       0.023
product_analytic_vertical_MotionController        0.0170      0.001     22.855      0.000       0.016       0.018
product_analytic_vertical_TVOutCableAccessory     0.0039      0.000     14.662      0.000       0.003       0.004
ItemType_MassMarket                              -0.0384      0.000   -142.961      0.000      -0.039      -0.038
==============================================================================
Omnibus:                   192226.827   Durbin-Watson:                   2.004
Prob(Omnibus):                  0.000   Jarque-Bera (JB):       7103151402.463
Skew:                          29.156   Prob(JB):                         0.00
Kurtosis:                    1511.871   Cond. No.                     5.68e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.68e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [66]:
vif_calc(input_data=X_train_rfe1)

,Features,VIF
0,const,"23,179.91"
6,NPSScore,3.30
4,Total Investment,3.24
2,product_mrp,1.60
13,ItemType_MassMarket,1.57
3,Discount_Percentage,1.15
5,Stock Index,1.13
11,product_analytic_vertical_MotionController,1.04
12,product_analytic_vertical_TVOutCableAccessory,1.03
8,product_analytic_vertical_GamingAccessoryKit,1.02


In [67]:
df_test[varlist1] = scaler.transform(df_test[varlist1])

In [68]:
# Split the 'df_test' set into X and y
y_test = df_test.pop('gmv')
X_test = df_test

In [69]:
col1

Index(['units', 'product_mrp', 'Discount_Percentage', 'gmvperunit',
       'Total Investment', 'NPS', 'Stock Index', 'NPSScore',
       'product_analytic_vertical_GameControlMount',
       'product_analytic_vertical_GamingAccessoryKit',
       'product_analytic_vertical_GamingSpeaker',
       'product_analytic_vertical_JoystickGamingWheel',
       'product_analytic_vertical_MotionController',
       'product_analytic_vertical_TVOutCableAccessory', 'ItemType_MassMarket'],
      dtype='object')

In [70]:
# Let's check the list 'col2' which had the 10 variables RFE had selected
X_test_rfe2 = X_test[col1]

In [71]:
X_train_rfe1.columns

Index(['const', 'units', 'product_mrp', 'Discount_Percentage',
       'Total Investment', 'Stock Index', 'NPSScore',
       'product_analytic_vertical_GameControlMount',
       'product_analytic_vertical_GamingAccessoryKit',
       'product_analytic_vertical_GamingSpeaker',
       'product_analytic_vertical_JoystickGamingWheel',
       'product_analytic_vertical_MotionController',
       'product_analytic_vertical_TVOutCableAccessory', 'ItemType_MassMarket'],
      dtype='object')

In [72]:
X_test_rfe2 = sm.add_constant(X_test_rfe2)

In [73]:
X_test_rfe2.columns

Index(['const', 'units', 'product_mrp', 'Discount_Percentage', 'gmvperunit',
       'Total Investment', 'NPS', 'Stock Index', 'NPSScore',
       'product_analytic_vertical_GameControlMount',
       'product_analytic_vertical_GamingAccessoryKit',
       'product_analytic_vertical_GamingSpeaker',
       'product_analytic_vertical_JoystickGamingWheel',
       'product_analytic_vertical_MotionController',
       'product_analytic_vertical_TVOutCableAccessory', 'ItemType_MassMarket'],
      dtype='object')

In [74]:
# Let's now drop the variables we had manually eliminated as well
#X_test_rfe2 =X_test_rfe2[col]
X_test = df_test
X_test_rfe2 = X_test_rfe2[X_train_rfe1.columns]

In [75]:
# Making predictions
y_pred = lm_3.predict(X_test_rfe2)

In [80]:
# r2_score for 6 variables
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.7051401873761362

In [76]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse = sqrt(mean_squared_error(y_test, y_pred))
print(rmse)

0.012079104721734645


In [77]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
import numpy as np
regr = LinearRegression() 

In [78]:
scores = cross_val_score(regr, X_train, y_train, cv = 5)    #cv is the number of folds, scores will give an array of scores

print(scores, np.mean(scores), np.std(scores))

#To get predictions (y_hat) and check them all in one using cross validation

predictions = cross_val_predict(regr, X_test, y_test, cv = 5)     #y_test is needed here in predictions to get scores for each fold of cv

accuracy = metrics.r2_score(y_test, predictions)  #this says the accuracy of the predictions from the best cv fold


#If this is good, continue to fit the model on the data


regr.fit(X_train, y_train)

y_hat = regr.predict(X_test)      #this gives me my predictions

regr.score(X_test, y_test)  

[0.58845787 0.64952768 0.74886083 0.69441254 0.82094399] 0.7004405831853119 0.08004271293962939


0.7281627552190086

In [79]:
accuracy

0.7283346211015738